In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from glob import glob
%matplotlib inline

In [2]:
folderLoc ='/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/downloads/Buoy/extractedGZ/WINDS/'
fileList = glob(folderLoc + '*_xrr_COARE3p5_2000_withRAIN_2hrMeanVar.nc')

In [3]:
# count = 0
# for fname in fileList:
#     print(fname.lstrip(folderLoc).rstrip('_xrr_COARE3p5_2000_withRAIN_2hrMeanVar.nc'))
#     ds = xr.open_dataset(fname)
#     ds['mean_WVEL'] = np.sqrt(ds['mean_U10N_x'] **2 + ds['mean_U10N_y']**2)
#     selectLabels = ['mean_WVEL', 'mean_WSPD', 'std_WSPD', 
#                 'std_cosWDIR' , 'std_sinWDIR', 
#                 'mean_SST', 'std_SST', 
#                 'mean_AIRT', 'std_AIRT',
#                 'mean_RELH', 'std_RELH',
#                 'mean_RAIN']
#     subDS = ds[selectLabels]
#     selectMask = ~np.isnan(subDS['mean_WVEL'].to_numpy())
#     subDS = subDS.isel(TIME=selectMask)
#     stdLabels = ['std_WSPD', 'std_cosWDIR', 'std_sinWDIR', 'std_SST', 'std_AIRT', 'std_RELH']
#     for label in stdLabels:
#         mask = np.isnan(subDS[label].to_numpy())
#         #print(label, np.sum(mask))
#         subDS[label] = xr.where(mask, 0.0, subDS[label])
#     data = subDS.to_dataframe()
#     if count == 0:
#         allDF = data
#     else:
#         allDF = pd.concat([allDF, data])
#     ds.close()
#     count+= 1
    
            

In [4]:
# allDF

In [5]:
# allDF.to_csv('allData.csv', index=False)

In [6]:
allDF = pd.read_csv('allData.csv')

In [8]:
len(allDF)

4227846

In [17]:
data = allDF
data['RAIN_LABEL'] = data['mean_RAIN'].apply(lambda x: 1 if x > 1 else 0)

# Features to use for classification
features = ['mean_WSPD', 'mean_SST', 'mean_AIRT', 'mean_RELH', 'std_WSPD', 'std_cosWDIR', 'std_sinWDIR', 'std_SST', 'std_AIRT', 'std_RELH']

# Split the dataset into features (X) and target (y)
X = data[features]
y = data['RAIN_LABEL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForest Classifier
classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9883309483543645

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    822417
           1       0.89      0.65      0.75     23153

    accuracy                           0.99    845570
   macro avg       0.94      0.83      0.87    845570
weighted avg       0.99      0.99      0.99    845570



In [18]:
falsePos = np.logical_and(y_test ==0, y_pred == 1)
falseNeg = np.logical_and(y_test ==1, y_pred == 0)

In [19]:
np.sum(falsePos)

1826

In [20]:
np.sum(falseNeg)

8041

In [21]:
len(y_test)

845570

In [22]:
(np.sum(falsePos) + np.sum(falseNeg))/845570*100

1.1669051645635489

In [23]:
import joblib
# from sklearn.ensemble import RandomForestClassifier
# # create RF
# rf = RandomForestClassifier()
# # fit on some data
# rf.fit(X, y)

# save
joblib.dump(classifier, "random_forest_rainDetection.joblib")

# # load
# loaded_rf = joblib.load("my_random_forest.joblib")

['random_forest_rainDetection.joblib']

In [7]:
2273/(145982 + 2273) * 100

1.5331692017132643

In [9]:
145982 + 2273

148255